In [1]:
user_input = 'I want to take a software development course, give me some recommendations'
rag_num_return = 10

rag_model_path = '../models/gte-base-en-v1.5'
rerank_model_path = "../models/gte-multilingual-reranker-base"
llm_model_name = "../models/Qwen2.5-0.5B-Finetuned"

# RAG

In [2]:
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

client = MilvusClient("dukies.db")


/Users/zihengs/myenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
I0000 00:00:1736008489.843167 5532294 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1736008489.851069 5532790 subchannel.cc:806] subchannel 0x32fa3f0d0 {address=unix:/var/folders/rn/5219x28n1bb9p3w3td_571000000gn/T/tmppn0fxy_2_dukies.db.sock, args={grpc.client_channel_factory=0x32c5f7dd0, grpc.default_authority=var%2Ffolders%2Frn%2F5219x28n1bb9p3w3td_571000000gn%2FT%2Ftmppn0fxy_2_dukies.db.sock, grpc.enable_retries=1, grpc.internal.channel_credentials=0x32c575d40, grpc.internal.client_channel_call_de

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f'using {device} for inferencing')

model = SentenceTransformer(rag_model_path, trust_remote_code=True)
model.to(device)

query_vectors = model.encode([user_input])

res = client.search(
    collection_name="rag_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=rag_num_return,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

res_list = [res[0][i]['entity']['text'] for i in range(len(res[0]))]
dist_list = [res[0][i]['distance'] for i in range(len(res[0]))]

using mps for inferencing


I0000 00:00:1736008495.319229 5532294 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [4]:
for res in res_list:
    print(res)

Course Code: ECE 651K
Course Name: Software Engineering
Description:
Teaches students about all steps of the software development lifecycle: requirements definition, design, development, testing, and maintenance. The course assumes students are skilled object-oriented programmers from prior courses, but will include a rapid introduction to Java. Students complete team-based semester-long software project which will progress through all phases of the software lifecycle. Taught at Duke Kunshan University in Kunshan, China. Prerequisite: ECE 551K.
Course Code: ECE 564
Course Name: Mobile Application Development
Description:
Explores mobile application development in the Apple Development Environment. Uses core software engineering pillars of Swift, Xcode, iOS & Xcode Cloud to learn how to create apps for Apple products. Focuses on iOS/iPhone, but Xcode also allows for exploration into the VisionPro, the Apple Watch, the iPad, and Apple TV. Real world context focused on common programming 

# Rerank

In [5]:
tokenizer = AutoTokenizer.from_pretrained(rerank_model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    rerank_model_path, trust_remote_code=True,
    torch_dtype=torch.float16
).to(device)
model.eval()

pairs = [[user_input, res_list[i]] for i in range(len(res_list))]

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()

selected_responses = [res_list[i] for i, score in enumerate(scores) if score > 0]


In [6]:
def remove_empty_lines(input_string):
    return '\n'.join(line for line in input_string.splitlines() if line.strip())
selected_responses = [remove_empty_lines(s) for s in selected_responses]

rag_str = '\n\n'.join([response for response in selected_responses])


final_prompt = f'''Answer the following questions as best you can. You have refer to the following context:

----------
{rag_str}
----------

Question: {user_input}
'''
print(final_prompt)

Answer the following questions as best you can. You have refer to the following context:

----------
Course Code: ECE 651K
Course Name: Software Engineering
Description:
Teaches students about all steps of the software development lifecycle: requirements definition, design, development, testing, and maintenance. The course assumes students are skilled object-oriented programmers from prior courses, but will include a rapid introduction to Java. Students complete team-based semester-long software project which will progress through all phases of the software lifecycle. Taught at Duke Kunshan University in Kunshan, China. Prerequisite: ECE 551K.

Course Code: ECE 564
Course Name: Mobile Application Development
Description:
Explores mobile application development in the Apple Development Environment. Uses core software engineering pillars of Swift, Xcode, iOS & Xcode Cloud to learn how to create apps for Apple products. Focuses on iOS/iPhone, but Xcode also allows for exploration into the

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    torch_dtype="auto",
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

messages = [
    {"role": "system", "content": "You are Dukies, a helpful advisor."},
    {"role": "user", "content": final_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)
with torch.no_grad():  # Disable gradient calculations for inference
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
    )
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

Based on the provided information, here are some recommendations for taking software development courses:

1. **ECE 651K** (Software Engineering): This course is designed specifically for students who are already proficient in object-oriented programming languages like C++ and have taken previous courses in ECE 551K. It covers all stages of the software development lifecycle, making it suitable for those with strong object-oriented programming skills.

2. **ECE 564**: This course focuses on creating mobile applications using the Apple Development Environment. While it requires familiarity with Swift and Xcode, it provides a practical approach to learning how to develop apps for various Apple devices, enhancing your problem-solving skills and understanding of software engineering principles.

Both courses align well with the context of developing software within a specific industry and provide relevant coursework experiences.
